In [1]:
import polars as pl
import sys
import yaml
import os

In [2]:
sys.path.append('../src/')
project_root = os.path.dirname(os.getcwd())

In [3]:
# Directorios para los archivos de parámetros y los datos
parameters_directory = os.path.join(project_root, 'src', 'parameters')
data_raw_directory = os.path.join(project_root, 'data', 'raw')
data_processed_directory = os.path.join(project_root, 'data', 'processed')
data_features_directory = os.path.join(project_root, 'data', 'features')

In [19]:
# Lista todos los archivos YAML en el directorio especificado
yaml_files = [f for f in os.listdir(parameters_directory) if f.endswith('.yml')]

# Diccionario para guardar los parámetros cargados
parameters = {}

# Carga cada archivo YAML
for yaml_file in yaml_files:
    with open(os.path.join(parameters_directory, yaml_file), 'r') as file:
        data = yaml.safe_load(file)
        key_name = f'parameters_{yaml_file.replace(".yml", "")}'
        parameters[key_name] = data

PROCESSING DATA

In [5]:
# Nodos
from functions.processing import (validate_tags_pl, 
                             validate_dtypes_pl, 
                             change_names_pl, 
                             change_dtype_pl,
                             delete_accents_pl,
                             standardize_binary_values_pl,
                             impute_missing_values_pl)

In [6]:
tag_dict_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['tag_dict_path'])
raw_data_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['raw_data_path'])

tag_dict = pl.read_excel(tag_dict_path)
data_raw = pl.read_csv(raw_data_path)

In [7]:
prueba1 = validate_tags_pl(data_raw, tag_dict)

INFO:functions.processing:Iniciando la validación de tags...
INFO:functions.processing:Tags validados!


In [8]:
prueba2 = validate_dtypes_pl(prueba1, tag_dict)

INFO:functions.processing:Iniciando la validación de tipos de datos...
INFO:functions.processing:Tipos de datos validados correctamente!


In [9]:
prueba3 = change_names_pl(prueba2, tag_dict)

INFO:functions.processing:Iniciando el cambio de nombres...
INFO:functions.processing:Nombres de columnas cambiados!


In [10]:
prueba4 = change_dtype_pl(prueba3, tag_dict)

INFO:functions.processing:Iniciando el cambio de tipos de datos...
INFO:functions.processing:Dtypes cambiados!


In [11]:
prueba5 = delete_accents_pl(prueba4)

INFO:functions.processing:Iniciando la eliminación de acentos...
INFO:functions.processing:Acentos eliminados!


In [12]:
prueba6 = standardize_binary_values_pl(prueba5, parameters['parameters_processing'])

INFO:functions.processing:Iniciando la estandarización de valores binarios...
INFO:functions.processing:Valores binarios estandarizados!


In [13]:
prueba7 = impute_missing_values_pl(prueba6)

INFO:functions.processing:Iniciando la imputación de valores faltantes...
INFO:functions.processing:Imputados valores nulos en la columna 'tipo_trabajo' con la moda: administrador negocio
INFO:functions.processing:Imputados valores nulos en la columna 'estado_civil' con la moda: casado
INFO:functions.processing:Imputados valores nulos en la columna 'educacion' con la moda: pregrado
INFO:functions.processing:Imputados valores nulos en la columna 'mora' con la mediana redondeada: 0
INFO:functions.processing:Imputados valores nulos en la columna 'posee_vivienda' con la mediana redondeada: 1
INFO:functions.processing:Imputados valores nulos en la columna 'posee_vehiculo' con la mediana redondeada: 0
INFO:functions.processing:Imputados valores nulos en la columna 'vlr_ingresos' con la mediana: 1502287.936143504
INFO:functions.processing:Imputación de valores faltantes completada!


FEATURING DATA

In [14]:
# Nodos
from functions.featuring import (new_features_pl,
                                 add_target_variable_pl)

In [15]:
processed_data_path = os.path.join(data_processed_directory, parameters['parameters_catalog']['processed_data_path'])

data_processed = pl.read_csv(processed_data_path)

In [16]:
prueba8 = new_features_pl(data_processed, parameters['parameters_featuring'])

INFO:functions.featuring:Iniciando la generación de nuevas características...
INFO:functions.featuring:Nuevas características generadas!


In [17]:
target_directory = os.path.join(project_root, 'data', 'processed')
target_path = os.path.join(target_directory, parameters['parameters_catalog']['target_column_path'])
target = pl.read_csv(target_path)
prueba9 = add_target_variable_pl(prueba8, target, parameters['parameters_featuring'])

INFO:functions.featuring:Añadiendo la variable objetivo al DataFrame...
INFO:functions.featuring:Variable objetivo añadida!
